In [1]:
import os
import csv

In [2]:
def extract_min_values_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Ensure file has at least two lines to read
        if len(lines) < 2:
            return None, None
        
        # Parse the two lines
        try:
            score_values = list(map(float, lines[0].split()))
            rmse_values = list(map(float, lines[1].split()))
            
            # Get the minimum values
            min_score = min(score_values)
            min_rmse = min(rmse_values)
            
            return min_score, min_rmse
        except ValueError:
            # Handle cases where conversion to float fails
            return None, None

In [3]:
def process_folders_in_directory(directory):
    # Initialize a dictionary to store results with folder names as keys
    results = {}

    # Initialize a list to store the file names (used as column headers)
    folder_names = []

    # Iterate through each folder in the specified directory
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        
        if os.path.isdir(folder_path):
            folder_names.append(folder)  # Add folder name
            
            # Initialize values for this folder
            folder_scores = {
                "FD001 Score": None, "FD001 RMSE": None,
                "FD002 Score": None, "FD002 RMSE": None,
                "FD003 Score": None, "FD003 RMSE": None,
                "FD004 Score": None, "FD004 RMSE": None
            }

            # Find the relevant files within this folder
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)

                # Check if the file starts with FD001, FD002, etc.
                if file_name.startswith("FD001") or file_name.startswith("FD002") or \
                   file_name.startswith("FD003") or file_name.startswith("FD004"):
                    
                    # Extract the minimum Score and RMSE
                    min_score, min_rmse = extract_min_values_from_file(file_path)

                    # Store the values in folder_scores
                    if min_score is not None and min_rmse is not None:
                        key_score = f"{file_name[:5]} Score"
                        key_rmse = f"{file_name[:5]} RMSE"
                        
                        folder_scores[key_score] = min_score
                        folder_scores[key_rmse] = min_rmse
            
            # Store the folder_scores under the folder name in results
            results[folder] = folder_scores

    # Write the results to a CSV file
    with open("output.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        # Create column headers, starting with Folder Name
        column_headers = ["Folder Name"] + list(folder_scores.keys())

        writer.writerow(column_headers)

        # Write data rows
        for folder in folder_names:
            row = [folder] + list(results[folder].values())
            writer.writerow(row)


In [4]:
# Specify your directory path here
directory_path = "./result"
process_folders_in_directory(directory_path)